In [ ]:
import os
if not os.environ.get("PREAMBLE_RUN", False):
    %run -i "../preamble.py" 2

In [18]:
import pandas as pd
import numpy as np

from src.constants import PATH_INTERMED_CHES_52_NAT_MEMBER, PATH_INTERMED_CHES_50_CHES_META, PATH_INTERMED_CHES_53_ENRICHED_MEMBER
from src.normalize_strings import normalize_list, normalize_name_strict

# set display options for the notebook
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

Now let's merge the previously created dataset of national party memberships with our CHES metadata parsed in the beginning., i.e. create a mapping between the memberships' party ids and the corresponding CHES party ids

In [ ]:
# load the data
original_nat_member_df = pd.read_parquet(PATH_INTERMED_CHES_52_NAT_MEMBER, engine='fastparquet')
original_ches_meta_df = pd.read_parquet(PATH_INTERMED_CHES_50_CHES_META, engine='fastparquet')

So let's first get a mapping between membership party ids and the CHES party ids

In [21]:
# work on copies
nat_member_df = original_nat_member_df.copy(deep=True)
ches_meta_df = original_ches_meta_df.copy(deep=True)
print(nat_member_df.shape[0])
display(nat_member_df.head(2))
print(ches_meta_df.shape[0])
display(ches_meta_df.head(2))

2287


,person_id,person_label,hasMembership,party_id,member_startDate,member_endDate,member_role,party_class,party_represents,party_temporal,party_label,party_all_labels,party_all_names
0,102886,Ildikó GÁLL-PELCZ,"[{'id': 'membership/102886-f-149915', 'type': ...",4068,2014-07-01,2017-08-31,MEMBER,NATIONAL_POLITICAL_GROUP,HUN,"{'id': 'time-period/20140701-20190701', 'type'...",Fidesz-KDNP,[Fidesz-KDNP],[Fidesz-Magyar Polgári Szövetség-Kereszténydem...
1,102886,Ildikó GÁLL-PELCZ,"[{'id': 'membership/102886-f-149915', 'type': ...",2753,2010-06-02,2014-06-30,MEMBER,NATIONAL_POLITICAL_GROUP,HUN,"{'id': 'time-period/20090714-20140630', 'type'...",Fidesz-KDNP,[Fidesz-KDNP],[Fidesz-Magyar Polgári Szövetség-Keresztény De...


481


,party_id,country_code,party_abbrev,party_name,party_name_en
0,102,BEL,[PS],[Parti Socialiste],[Socialist Party]
1,103,BEL,"[SP, SP/SPA, SPA, Vooruit, Vooruit/SP]","[Vooruit, Socialistische Partij, Socialistisch...","[Onward, Socialist Party, Socialist Party Diff..."


Do some preprocessing / harmonization and normalize party names to ensure better comparability

In [22]:
# first harmonize column names
nat_member_df = nat_member_df.add_suffix("_memb").rename(columns={"party_represents_memb" : "country_code_memb"})
ches_meta_df = ches_meta_df.add_suffix("_ches")

In [23]:
# ensure that country codes are in ISO format
eu_country_codes_iso = ["AUT", "BEL", "BGR", "HRV", "CYP", "CZE", "DNK", "EST", "FIN", "FRA", "DEU", "GRC", "HUN", "IRL", "ITA", "LVA", "LTU", "LUX", "MLT", "NLD", "POL", "PRT", "ROU", "SVK", "SVN", "ESP", "SWE", "GBR"]
invalid_iso_ches = ches_meta_df.loc[~ches_meta_df["country_code_ches"].isin(eu_country_codes_iso), "country_code_ches"]
invalid_iso_memb = nat_member_df.loc[~nat_member_df["country_code_memb"].isin(eu_country_codes_iso), "country_code_memb"]
assert invalid_iso_ches.empty
assert invalid_iso_memb.empty

# ensure that party_ids are strings
bad_id_rows_ches = ches_meta_df[~ches_meta_df["party_id_ches"].dropna().apply(type).eq(str)]
bad_id_rows_memb = nat_member_df[~nat_member_df["party_id_memb"].dropna().apply(type).eq(str)]
ches_meta_df["party_id_ches"] = ches_meta_df["party_id_ches"].astype("string")
nat_member_df["party_id_memb"] = nat_member_df["party_id_memb"].astype("string")
bad_id_rows_ches = ches_meta_df[~ches_meta_df["party_id_ches"].dropna().apply(type).eq(str)]
bad_id_rows_memb = nat_member_df[~nat_member_df["party_id_memb"].dropna().apply(type).eq(str)]
assert bad_id_rows_ches.empty
assert bad_id_rows_memb.empty

In [ ]:
#display(nat_member_df.head())
#display(ches_meta_df.head())

In [24]:
# now apply normalization of party labels and names
ches_meta_df = ches_meta_df.assign(
    label_norm_ches=ches_meta_df["party_abbrev_ches"].apply(normalize_list),
    name_norm_ches=ches_meta_df.apply(
        lambda r: normalize_list(r["party_name_ches"])       # fuse native and english names
                | normalize_list(r["party_name_en_ches"]),
        axis=1
    )
)

nat_member_df = nat_member_df.assign(
    label_norm_memb=nat_member_df["party_all_labels_memb"].apply(normalize_list),    # column already contains all possible labels
    name_norm_memb=nat_member_df["party_all_names_memb"].apply(normalize_list)       # same for names
)
print("Normalization finished!")
display(ches_meta_df.head(2))
display(nat_member_df.head(2))

Normalization finished!


,party_id_ches,country_code_ches,party_abbrev_ches,party_name_ches,party_name_en_ches,label_norm_ches,name_norm_ches
0,102,BEL,[PS],[Parti Socialiste],[Socialist Party],{ps},"{parti socialiste, socialist party}"
1,103,BEL,"[SP, SP/SPA, SPA, Vooruit, Vooruit/SP]","[Vooruit, Socialistische Partij, Socialistisch...","[Onward, Socialist Party, Socialist Party Diff...","{sp, spa, sp/spa, vooruit, vooruit/sp}","{socialist party differently, socialistische p..."


,person_id_memb,person_label_memb,hasMembership_memb,party_id_memb,member_startDate_memb,member_endDate_memb,member_role_memb,party_class_memb,country_code_memb,party_temporal_memb,party_label_memb,party_all_labels_memb,party_all_names_memb,label_norm_memb,name_norm_memb
0,102886,Ildikó GÁLL-PELCZ,"[{'id': 'membership/102886-f-149915', 'type': ...",4068,2014-07-01,2017-08-31,MEMBER,NATIONAL_POLITICAL_GROUP,HUN,"{'id': 'time-period/20140701-20190701', 'type'...",Fidesz-KDNP,[Fidesz-KDNP],[Fidesz-Magyar Polgári Szövetség-Kereszténydem...,{fidesz-kdnp},{fidesz-magyar polgari szovetseg-keresztenydem...
1,102886,Ildikó GÁLL-PELCZ,"[{'id': 'membership/102886-f-149915', 'type': ...",2753,2010-06-02,2014-06-30,MEMBER,NATIONAL_POLITICAL_GROUP,HUN,"{'id': 'time-period/20090714-20140630', 'type'...",Fidesz-KDNP,[Fidesz-KDNP],[Fidesz-Magyar Polgári Szövetség-Keresztény De...,{fidesz-kdnp},{fidesz-magyar polgari szovetseg-kereszteny de...


In [25]:
# get rid of unnecessary columns, that we don't need for the initial mapping
ches_for_map = ches_meta_df.copy(deep=True)[["party_id_ches", "country_code_ches", "label_norm_ches", "name_norm_ches"]]
nat_member_for_map = nat_member_df.copy(deep=True)[["party_id_memb", "country_code_memb", "party_label_memb", "label_norm_memb", "name_norm_memb"]]
display(ches_for_map.head(5))
display(nat_member_for_map.head(5))

,party_id_ches,country_code_ches,label_norm_ches,name_norm_ches
0,102,BEL,{ps},"{parti socialiste, socialist party}"
1,103,BEL,"{sp, spa, sp/spa, vooruit, vooruit/sp}","{socialist party differently, socialistische p..."
2,104,BEL,{ecolo},{ecolo}
3,105,BEL,"{groen, agalev}","{groen!, green!, agalev}"
4,106,BEL,"{mr, prl/mr, prl}","{liberal reformist party, mouvement reformateu..."


,party_id_memb,country_code_memb,party_label_memb,label_norm_memb,name_norm_memb
0,4068,HUN,Fidesz-KDNP,{fidesz-kdnp},{fidesz-magyar polgari szovetseg-keresztenydem...
1,2753,HUN,Fidesz-KDNP,{fidesz-kdnp},{fidesz-magyar polgari szovetseg-kereszteny de...
2,6761,BGR,GERB,"{gerb, герб}",{citizens for european development of bulgaria...
3,5180,BGR,GERB,"{gerb, герб}",{citizens for european development of bulgaria...
4,3993,BGR,GERB,"{gerb, герб}",{citizens for european development of bulgaria...


In [9]:
## drop party duplicates in nat_member_normalized as we only care about the mapping between party ids for now
#amount_dup_drop = nat_member_normalized.shape[0]
#nat_member_normalized = nat_member_normalized.drop_duplicates(subset=["party_id_memb", "country_code_memb", "party_label_memb"])
#print("Size of ches: ", ches_normalized.shape[0])
#print("Size of nat_member_normalized", nat_member_normalized.shape[0])
#print("Amout of duplicate entries in nat_member_normalized dropped: ", amount_dup_drop - nat_member_normalized.shape[0])

Perform the initial merge, where a match happens in one of the following cases:
- country code is the same for both the EP/membership party and the CHES party
- either there is an overlap between the normalized set of possible party labels of both EP/membership and CHES party or
the same holds for the party names respectively

In [26]:
# perform the initial merge with overlap condition

# create merge candidates based on equal country code
tmp = nat_member_for_map.merge(
    ches_for_map,
    left_on="country_code_memb",
    right_on="country_code_ches",
    how="left",
    suffixes=("_memb", "_ches")
)

def overlaps(a, b):
    return bool(a and b and (a & b))  # assumes sets; adapt if lists

# apply overlap condition
mask = tmp.apply(
    lambda r: overlaps(r["label_norm_memb"], r["label_norm_ches"]) or
              overlaps(r["name_norm_memb"],  r["name_norm_ches"]), # or
                #overlaps(r["label_norm_memb"],  r["name_norm_ches"]) or
                #overlaps(r["name_norm_memb"],  r["label_norm_ches"]),
    axis=1
)

# eliminate wrong/invalid merge candidates (i.e. party pairs of same country but unequal label/name)
merged = tmp[mask]

merged_copy = merged.copy(deep=True)
print(merged_copy.shape[0])
merged_copy = merged_copy.drop_duplicates(subset=["party_id_ches", "party_id_memb"], keep="first")
print(merged_copy.shape[0])
display(merged_copy.sort_values(by=["party_id_ches"], ascending=False).head(5))
#display(merged_copy.loc[merged_copy["party_id_ches"].isna()].sort_values(by=["country_code_ches"], ascending=False).head(5))

1782
605


,party_id_memb,country_code_memb,party_label_memb,label_norm_memb,name_norm_memb,party_id_ches,country_code_ches,label_norm_ches,name_norm_ches
19788,6324,ITA,E.Verde,"{e.verde, europa verde}",{europa verde},850,ITA,{ev},"{europa verde, green europe}"
38579,3844,ITA,NCD,{ncd},{nuovo centrodestra},848,ITA,{ncd},"{new centre-right, nuovo centrodestra}"
20134,6674,ITA,M5S,{m5s},{movimento 5 stelle},845,ITA,"{ms5, m5s}","{movimento cinque stelle, five star movement}"
1485,5268,ITA,M5S,{m5s},{movimento 5 stelle},845,ITA,"{ms5, m5s}","{movimento cinque stelle, five star movement}"
1524,4083,ITA,M5S,{m5s},{movimento 5 stelle},845,ITA,"{ms5, m5s}","{movimento cinque stelle, five star movement}"


In [27]:
# quick sanity check, names must match row-wise
merged_copy["same_country"] = merged_copy["country_code_ches"] == merged_copy["country_code_memb"]
print(merged_copy["same_country"].sum())

605


In [28]:
# extract the membership to CHES mapping (of party ids)
initial_ID_mapping_memb2CHES = merged[["party_id_ches", "party_id_memb"]]#.drop_duplicates(subset=["party_id_memb", "party_id_ches"]).reset_index(drop=True)
print(initial_ID_mapping_memb2CHES.shape[0])
initial_ID_mapping_memb2CHES = initial_ID_mapping_memb2CHES.drop_duplicates(subset=["party_id_memb", "party_id_ches"]).reset_index(drop=True)
print("Initial amount of (unique) ID mappings found:", initial_ID_mapping_memb2CHES.shape[0])
#display(initial_ID_mapping_memb2CHES.head(2))

1782
Initial amount of (unique) ID mappings found: 605


In [29]:
# quick smell check if the mapping is unique indeed
print(initial_ID_mapping_memb2CHES.loc[initial_ID_mapping_memb2CHES.duplicated(subset="party_id_memb", keep=False).values])
print(initial_ID_mapping_memb2CHES.loc[initial_ID_mapping_memb2CHES.duplicated("party_id_ches").values].shape[0])
# no apparently there is an ambiguous mapping for MEP "Franc Bogovič"
display(nat_member_df.loc[nat_member_df["party_id_memb"].isin(["4282", "5411"])])
display(ches_meta_df.loc[ches_meta_df["party_id_ches"].isin(["2904", "2919"])])

# NOTE
#  Apparently there a two ids in CHES for the same party SLS (Slovenian People's Party) [see here](https://en.wikipedia.org/wiki/Slovenian_People%27s_Party), namely 2904 and 2919
# but for id 2919 we only have one datapoint (in 2024) and for id 2904 we have datapoints on ('02, '06, '10, and '14) therefore merge these two IDs together in CHES ! befor SUPERFINAL MERGE!!!!
# CONSEQUENCE (1) : Party with ID "2919" was removed from CHES_META -> DONE
# CONSEQUENCE (2) : MERGE Replace ID of party "2919" with "2904" to merge the parties together -> DONE

Empty DataFrame
Columns: [party_id_ches, party_id_memb]
Index: []
405


,person_id_memb,person_label_memb,hasMembership_memb,party_id_memb,member_startDate_memb,member_endDate_memb,member_role_memb,party_class_memb,country_code_memb,party_temporal_memb,party_label_memb,party_all_labels_memb,party_all_names_memb,label_norm_memb,name_norm_memb
709,125004,Franc BOGOVIČ,"[{'id': 'membership/125004-f-152357', 'type': ...",4282,2014-07-01,2019-07-01,MEMBER,NATIONAL_POLITICAL_GROUP,SVN,"{'id': 'time-period/20140701-20190701', 'type'...",SLS,[SLS],[Slovenska ljudska stranka],{sls},{slovenska ljudska stranka}
710,125004,Franc BOGOVIČ,"[{'id': 'membership/125004-f-152357', 'type': ...",5411,2019-07-02,2024-07-15,MEMBER,NATIONAL_POLITICAL_GROUP,SVN,"{'id': 'time-period/20190702-20240715', 'type'...",SLS,[SLS],[Slovenska ljudska stranka],{sls},{slovenska ljudska stranka}


,party_id_ches,country_code_ches,party_abbrev_ches,party_name_ches,party_name_en_ches,label_norm_ches,name_norm_ches
426,2904,SVN,"[SLS, SLS-SKD, SLS-SMS]",[Slovenska ljudska stranka],[Slovenian People’s Party–Slovenian Christian ...,"{sls, sls-sms, sls-skd}",{slovenian people’s party–slovenian christian ...


**NOTE**
Apparently there a two ids in CHES for the same party SLS (Slovenian People's Party) [see here](https://en.wikipedia.org/wiki/Slovenian_People%27s_Party), namely 2904 and 2919
but for id 2919 we only have one datapoint (in 2024) and for id 2904 we have datapoints on ('02, '06, '10, and '14) therefore merge these two IDs together in CHES ! befor SUPERFINAL MERGE!!!!
- CONSEQUENCE (1) : Party with ID "2919" was removed from CHES_META -> DONE
- CONSEQUENCE (2) : MERGE Replace ID of party "2919" with "2904" to merge the parties together -> ALREADY DONE TOO (-> here its therefore also removed already!)

**SANITY CHECK** Lets briefly investigate which CHES party was not matched to any membership party

In [30]:
unmatched_ches_parties = ches_meta_df.loc[
    ~ches_meta_df["party_id_ches"].isin(initial_ID_mapping_memb2CHES["party_id_ches"])
]
print("Amount of initial unmatched CHES parties: ", unmatched_ches_parties.shape[0])
display(unmatched_ches_parties.head(20))

Amount of initial unmatched CHES parties:  281


,party_id_ches,country_code_ches,party_abbrev_ches,party_name_ches,party_name_en_ches,label_norm_ches,name_norm_ches
9,111,BEL,"[DeFl, DéFI, FDF]","[Front Démocratique des Francophones, Démocrat...","[Francophone Democratic Front, Christian Democ...","{defi, defl, fdf}","{francophone democratic front, democrate feder..."
12,114,BEL,[ID21],[ID21],[ID21],{id21},{id21}
13,115,BEL,[FN],[Front National],[National Front],{fn},"{front national, national front}"
15,118,BEL,"[SLP, Spirit]","[Sociaal-Liberale Partij, Spirit]","[Social-Liberal Party, Spirit]","{slp, spirit}","{social-liberal party, spirit, sociaal-liberal..."
16,119,BEL,"[PVDA, PVDA-PTB]","[Partij van de Arbeid van België, Parti du Tra...",[Workers’ Party of Belgium],"{pvda-ptb, pvda}","{partij van de arbeid van belgie, parti du tra..."
17,120,BEL,[PP],[Parti Populaire],[People’s Party],{pp},"{parti populaire, people’s party}"
18,201,DNK,"[S, SD]",[Socialdemokraterne],[Social Democrats],"{s, sd}","{socialdemokraterne, social democrats}"
20,203,DNK,"[DKF, KF]",[Det Konservative Folksparti],[Conservative People’s Party],"{kf, dkf}","{det konservative folksparti, conservative peo..."
21,204,DNK,[CD],[Centrum-Demokraterne],[Centre Democrats],{cd},"{centre democrats, centrum-demokraterne}"
23,210,DNK,[KRF],[Kristeligt Folkeparti],[Christian People’s Party],{krf},"{christian people’s party, kristeligt folkeparti}"


It seems these CHES parties are not reflected in our PLS migration dataset. But more importantly is the question, which parties of our membership dataset did not get matched to their corresponding CHES party (either because there exists no
valid counterpart or because of matching problems, e.g. misspelling or ambiguous labels/names)?

In [31]:
# investigate the missing matches of memberships in the initial mapping to CHES
initial_unmatched_memb_rows = nat_member_df.loc[
    ~nat_member_df["party_id_memb"].isin(initial_ID_mapping_memb2CHES["party_id_memb"])
]

print("Initial amount of unmatched 'rows' in membership dataset after mapping: ", initial_unmatched_memb_rows.shape[0])
#display(unmatched_memb_parties.head(20))

##
##

# drop "independent" entries and their variants (ie. MEP was not party of any party for the given time period),
# also omit entries with value "-" in column "party_label_memb"
# NOTE: "DVD", "Droite Indépendante" refers to right wing independent politicians (france)
independent_labels = ["Indépendant", "Ind.", "Independent", "-", "Independiente", "s.e.", "Sans parti", "DVD", "Droite Indépendante", "Bezpartyjny", "niezależny"]
print("Amount of dropped rows because of removing 'independent':", initial_unmatched_memb_rows[initial_unmatched_memb_rows["party_label_memb"].isin(independent_labels)].shape[0]) # sanity check how many are we dropping?
initial_unmatched_memb_rows = initial_unmatched_memb_rows[~initial_unmatched_memb_rows["party_label_memb"].isin(independent_labels)]

print("Initial amount of unmatched 'rows' in membership dataset after mapping and removing 'independent': ", initial_unmatched_memb_rows.shape[0])
#display(unmatched_memb_parties.head(20))

##
##

# also drop any parties for which the unmatched membership ended before 2014 anyway (since our migration datasets starts from there)
initial_unmatched_memb_rows["member_endDate_memb"] = pd.to_datetime(initial_unmatched_memb_rows["member_endDate_memb"], errors="coerce") # ensure datetime
cutoff = pd.Timestamp("2014-01-01")
cutoff_mask_drop = initial_unmatched_memb_rows["member_endDate_memb"].notna() & (
    initial_unmatched_memb_rows["member_endDate_memb"] < cutoff
)
print("Amount of dropped rows because membership ended before 2014:", cutoff_mask_drop.sum()) # sanity check how many are we dropping?

initial_unmatched_memb_rows_final = initial_unmatched_memb_rows.loc[~cutoff_mask_drop].copy(deep=True)
print(
    "Initial amount of unmatched 'rows' in membership dataset after mapping, removing 'independent', and 2014-cutoff:",
    initial_unmatched_memb_rows_final.shape[0]
)

# also drop duplicates as we are only interested in distinct parties (i.e. unique party_id_memb's) for now
initial_unmatched_memb_rows_final = initial_unmatched_memb_rows_final.drop_duplicates(subset=["party_id_memb", "country_code_memb", "party_label_memb"]) #"party_all_labels", "party_all_names"])
print(
    "Initial amount of unmatched parties (ie. party_id_memb's) in membership dataset after mapping, removing 'independent', 2014-cutoff, and removing duplicates:",
    initial_unmatched_memb_rows_final.shape[0]
)
display(initial_unmatched_memb_rows_final.sort_values(by=["member_endDate_memb", "country_code_memb", "party_label_memb"]).head())

Initial amount of unmatched 'rows' in membership dataset after mapping:  505
Amount of dropped rows because of removing 'independent': 122
Initial amount of unmatched 'rows' in membership dataset after mapping and removing 'independent':  383
Amount of dropped rows because membership ended before 2014: 69
Initial amount of unmatched 'rows' in membership dataset after mapping, removing 'independent', and 2014-cutoff: 314
Initial amount of unmatched parties (ie. party_id_memb's) in membership dataset after mapping, removing 'independent', 2014-cutoff, and removing duplicates: 184


Investigate the missing Membership parties -> perform a manual semi-automatic (LLM-assisted) review of the missing memberships/parties left.
The resulting mapping of the parties for which a mapping could be resolved (this was done/checked manually for correctness):

In [32]:
# mapping that contains missing memb parties (values) with corresponding ches party id (keys)
missing_mapping_ches_to_memb = [
    # AUSTRIA
    ("1304", "5165"),

    # BELGIUM
    ("103","2973"),
    ("103","3985"),
    ("103","5172"),

    # BULGARIA
    ("2005", "4233"),
    ("2005", "5374"),

    # CYPRUS
    ("4003", "4001"),
    ("4003", "6765"),
    ("4003", "2682"),
    ("4003", "5188"),


    # CZECH REPUBLIC
    ("2114", "5524"),
    ("2109", "4212"),
    ("2114", "6862"),
    ("2109", "6793"),
    ("2109", "5354"),


    # GERMANY
    ("306", "5199"),
    ("304", "5197"),
    ("304", "6773"),
    ("306", "6775"),
    ("306", "2662"),
    ("306", "4012"),
    ("304", "2656"),
    ("304", "4010"),
    ("304", "1373"),

    # DENMARK
    ("203", "4016"),
    ("213", "5525"),
    ("203", "2696"),
    ("203", "5203"),

    # SPAIN
    ("524", "5399"),
    ("524", "6812"),
    ("519", "6816"),
    ("505", "2706"),
    ("505", "4872"),
    ("524", "4259"),
    ("519", "5413"),

    # FRANCE
    ("626", "5015"),
    ("605", "5225"),
    ("626", "5502"),
    ("603", "5040"),
    ("630", "6413"),
    ("605", "6953"),
    ("626", "5468"),
    ("605", "2956"),
    ("605", "4038"),

    # UK
    ("1110", "5073"),
    ("1104", "5236"),
    ("1104", "2738"),
    ("1106", "2740"),
    ("1106", "4286"),
    ("707", "2742"), # SIN FEIN -> IRISH
    ("707", "4250"), # SIN FEIN -> IRISH
    ("707", "5390"), # SIN FEIN -> IRISH
    ("1110", "5506"),
    ("1104", "4050"),
    ("1106", "5412"),

    # GREECE
    ("401", "5591"),
    ("417", "4923"),
    ("401", "4271"),
    ("402", "6730"),
    ("402", "4055"),
    ("402", "5241"),
    ("402", "5404"),


    # CROATIA
    ("3106", "4220"),
    ("3106", "5361"),

    # HUNGARY
    ("2310", "4165"),
    ("2314", "5527"),

    # IRELAND
    ("703", "2760"),

    # ITALY
    ("849", "6449"),
    ("849", "6478"),
    ("844", "3823"),
    ("844", "3890"),
    ("849", "6093"),
    ("815", "3842"),

    # LATVIA
    ("2410", "4104"),
    ("2406", "6685"),
    ("2406", "5288"),
    ("2406", "2797"),
    ("2406", "4103"),

    # POLAND
    ("2616", "6700"),
    ("2603", "7979"),
    ("2616", "5309"),
    ("2619", "5028"),

    # ROMANIA
    ("2712", "5023"),

    # SLOVAKIA
    ("2813", "4245"),
    ("2814", "4246"),
    ("2820", "6458"),
    ("2820", "6455"),

    # SLOVENIA
    ("2905", "4153"),
    ("2905", "2981"),
    ("2910", "2982"),
    ("2905", "3896"),

    # SWEDEN
    ("1602", "5328"),
    ("1602", "6712")

]

missing_matches_mapping_df = pd.DataFrame(
    missing_mapping_ches_to_memb, columns=["party_id_ches", "party_id_memb"])

In [33]:
# add the mapping of the missing matches to the existing mapping
final_ID_mapping_memb2CHES = pd.concat([initial_ID_mapping_memb2CHES, missing_matches_mapping_df], ignore_index=True)
print(final_ID_mapping_memb2CHES.shape[0])
display(final_ID_mapping_memb2CHES.loc[final_ID_mapping_memb2CHES.duplicated("party_id_memb").values])
print(final_ID_mapping_memb2CHES.drop_duplicates(["party_id_ches", "party_id_memb"]).shape[0])
print(final_ID_mapping_memb2CHES.drop_duplicates(["party_id_memb"]).shape[0])
#display(final_mapping_ches2memb)
print("Final amount of unique mappings found:", final_ID_mapping_memb2CHES.shape[0])

698


,party_id_ches,party_id_memb


698
698
Final amount of unique mappings found: 698


Finally, use this mapping of party ids between ches and membership dataset to merge the two together

In [34]:
# mapping table that contains the mapping between source (memb) and target (CHES) ID
map_df = final_ID_mapping_memb2CHES.copy(deep=True).reset_index(drop=True)
#display(map_df.head(2))

# the left dataframe (membership dataset) we want to enrich with CHES
left_df = nat_member_df.copy(deep=True).reset_index(drop=True)
display(left_df.head(2))

# the right dataframe (CHES) we want to merge with left
right_df = ches_meta_df.copy(deep=True).reset_index(drop=True)
display(right_df.head(2))

,person_id_memb,person_label_memb,hasMembership_memb,party_id_memb,member_startDate_memb,member_endDate_memb,member_role_memb,party_class_memb,country_code_memb,party_temporal_memb,party_label_memb,party_all_labels_memb,party_all_names_memb,label_norm_memb,name_norm_memb
0,102886,Ildikó GÁLL-PELCZ,"[{'id': 'membership/102886-f-149915', 'type': ...",4068,2014-07-01,2017-08-31,MEMBER,NATIONAL_POLITICAL_GROUP,HUN,"{'id': 'time-period/20140701-20190701', 'type'...",Fidesz-KDNP,[Fidesz-KDNP],[Fidesz-Magyar Polgári Szövetség-Kereszténydem...,{fidesz-kdnp},{fidesz-magyar polgari szovetseg-keresztenydem...
1,102886,Ildikó GÁLL-PELCZ,"[{'id': 'membership/102886-f-149915', 'type': ...",2753,2010-06-02,2014-06-30,MEMBER,NATIONAL_POLITICAL_GROUP,HUN,"{'id': 'time-period/20090714-20140630', 'type'...",Fidesz-KDNP,[Fidesz-KDNP],[Fidesz-Magyar Polgári Szövetség-Keresztény De...,{fidesz-kdnp},{fidesz-magyar polgari szovetseg-kereszteny de...


,party_id_ches,country_code_ches,party_abbrev_ches,party_name_ches,party_name_en_ches,label_norm_ches,name_norm_ches
0,102,BEL,[PS],[Parti Socialiste],[Socialist Party],{ps},"{parti socialiste, socialist party}"
1,103,BEL,"[SP, SP/SPA, SPA, Vooruit, Vooruit/SP]","[Vooruit, Socialistische Partij, Socialistisch...","[Onward, Socialist Party, Socialist Party Diff...","{sp, spa, sp/spa, vooruit, vooruit/sp}","{socialist party differently, socialistische p..."


In [35]:
enriched_memberships_df = (
    left_df
        # add party_id_ches via map_df, can have many rows match to one right "m:1"
        .merge(map_df, how="left", on="party_id_memb", validate="m:1")
        # right_df should be unique per party_id_ches -> "m:1" merge
        .merge(right_df, how="left", on="party_id_ches", validate="m:1")
)
display(enriched_memberships_df.head(5))

,person_id_memb,person_label_memb,hasMembership_memb,party_id_memb,member_startDate_memb,member_endDate_memb,member_role_memb,party_class_memb,country_code_memb,party_temporal_memb,party_label_memb,party_all_labels_memb,party_all_names_memb,label_norm_memb,name_norm_memb,party_id_ches,country_code_ches,party_abbrev_ches,party_name_ches,party_name_en_ches,label_norm_ches,name_norm_ches
0,102886,Ildikó GÁLL-PELCZ,"[{'id': 'membership/102886-f-149915', 'type': ...",4068,2014-07-01,2017-08-31,MEMBER,NATIONAL_POLITICAL_GROUP,HUN,"{'id': 'time-period/20140701-20190701', 'type'...",Fidesz-KDNP,[Fidesz-KDNP],[Fidesz-Magyar Polgári Szövetség-Kereszténydem...,{fidesz-kdnp},{fidesz-magyar polgari szovetseg-keresztenydem...,2302,HUN,"[Fidesz, Fidesz-KDNP, Fidesz-M]","[Fidesz—Magyar Polgári Szövetség, Fidesz—Keres...","[Fidesz—Hungarian Civic Union, Fidesz—Christia...","{fidesz, fidesz-kdnp, fidesz-m}","{fidesz—keresztenydemokrata neppart, fidesz—hu..."
1,102886,Ildikó GÁLL-PELCZ,"[{'id': 'membership/102886-f-149915', 'type': ...",2753,2010-06-02,2014-06-30,MEMBER,NATIONAL_POLITICAL_GROUP,HUN,"{'id': 'time-period/20090714-20140630', 'type'...",Fidesz-KDNP,[Fidesz-KDNP],[Fidesz-Magyar Polgári Szövetség-Keresztény De...,{fidesz-kdnp},{fidesz-magyar polgari szovetseg-kereszteny de...,2302,HUN,"[Fidesz, Fidesz-KDNP, Fidesz-M]","[Fidesz—Magyar Polgári Szövetség, Fidesz—Keres...","[Fidesz—Hungarian Civic Union, Fidesz—Christia...","{fidesz, fidesz-kdnp, fidesz-m}","{fidesz—keresztenydemokrata neppart, fidesz—hu..."
2,107212,Andrey NOVAKOV,"[{'id': 'membership/107212-f-173038', 'type': ...",6761,2024-07-16,None,MEMBER,NATIONAL_POLITICAL_GROUP,BGR,"{'id': 'time-period/20240716', 'type': 'Period...",GERB,"[ГЕРБ, GERB]","[ГЕРБ, Citizens for European Development of Bu...","{gerb, герб}",{citizens for european development of bulgaria...,2010,BGR,[GERB],[Grazhdani za evropeysko razvitie na Bălgariya],[Citizens for European Development of Bulgaria],{gerb},{grazhdani za evropeysko razvitie na balgariya...
3,107212,Andrey NOVAKOV,"[{'id': 'membership/107212-f-173038', 'type': ...",5180,2019-07-02,2024-07-15,MEMBER,NATIONAL_POLITICAL_GROUP,BGR,"{'id': 'time-period/20190702-20240715', 'type'...",GERB,"[ГЕРБ, GERB]","[ГЕРБ, Citizens for European Development of Bu...","{gerb, герб}",{citizens for european development of bulgaria...,2010,BGR,[GERB],[Grazhdani za evropeysko razvitie na Bălgariya],[Citizens for European Development of Bulgaria],{gerb},{grazhdani za evropeysko razvitie na balgariya...
4,107212,Andrey NOVAKOV,"[{'id': 'membership/107212-f-173038', 'type': ...",3993,2014-11-24,2019-07-01,MEMBER,NATIONAL_POLITICAL_GROUP,BGR,"{'id': 'time-period/20140701-20190701', 'type'...",GERB,"[ГЕРБ, GERB]","[ГЕРБ, Citizens for European Development of Bu...","{gerb, герб}",{citizens for european development of bulgaria...,2010,BGR,[GERB],[Grazhdani za evropeysko razvitie na Bălgariya],[Citizens for European Development of Bulgaria],{gerb},{grazhdani za evropeysko razvitie na balgariya...


In [36]:
#display(enriched_memberships_df.loc[enriched_memberships_df["party_id_ches"].isna()].sort_values(by=["country_code_memb", "party_label_memb"]))
print(enriched_memberships_df.loc[enriched_memberships_df["party_id_ches"].isna()].shape[0])

314


Sanity Check again, how many unmatched memberships we have now!

In [37]:
# investigate the missing matches of memberships in the final mapping to CHES
final_unmatched_memb_rows = nat_member_df.loc[
    ~nat_member_df["party_id_memb"].isin(final_ID_mapping_memb2CHES["party_id_memb"])
]

print("Final amount of unmatched 'rows' in membership dataset after mapping: ", final_unmatched_memb_rows.shape[0])
#display(final_unmatched_memb_rows.head(20))

##
##

# drop "independent" entries and their variants (ie. MEP was not party of any party for the given time period),
# also omit entries with value "-" in column "party_label_memb"
# NOTE: "DVD", "Droite Indépendante" refers to right wing independent politicians (france)
independent_labels = ["Indépendant", "Ind.", "Independent", "-", "Independiente", "s.e.", "Sans parti", "DVD", "Droite Indépendante", "Bezpartyjny", "niezależny", "Niezależny", "Bezpartyjna"]
print("Amount of dropped rows because of removing 'independent':", final_unmatched_memb_rows[final_unmatched_memb_rows["party_label_memb"].isin(independent_labels)].shape[0]) # sanity check how many are we dropping?
final_unmatched_memb_rows = final_unmatched_memb_rows[~final_unmatched_memb_rows["party_label_memb"].isin(independent_labels)]

print("Final amount of unmatched 'rows' in membership dataset after mapping and removing 'independent': ", final_unmatched_memb_rows.shape[0])
#display(unmatched_memb_parties.head(20))

##
##

# also drop any parties for which the unmatched membership ended before 2014 anyway (since our migration datasets starts from there)
final_unmatched_memb_rows["member_endDate_memb"] = pd.to_datetime(final_unmatched_memb_rows["member_endDate_memb"], errors="coerce") # ensure datetime
cutoff = pd.Timestamp("2014-01-01")
cutoff_mask_drop = final_unmatched_memb_rows["member_endDate_memb"].notna() & (
    final_unmatched_memb_rows["member_endDate_memb"] < cutoff
)
print("Amount of dropped rows because membership ended before 2014:", cutoff_mask_drop.sum()) # sanity check how many are we dropping?

final_unmatched_memb_rows_final = final_unmatched_memb_rows.loc[~cutoff_mask_drop].copy(deep=True)
print(
    "Final amount of unmatched 'rows' in membership dataset after mapping, removing 'independent', and 2014-cutoff:",
    final_unmatched_memb_rows_final.shape[0]
)

##
##

# also drop duplicates as we are only interested in distinct parties (i.e. unique party_id_memb's) for now
final_unmatched_memb_rows_final = final_unmatched_memb_rows_final.drop_duplicates(subset=["party_id_memb", "country_code_memb", "party_label_memb"]) #"party_all_labels", "party_all_names"])
print(
    "Final amount of unmatched parties (ie. party_id_memb's) in membership dataset after mapping, removing 'independent', 2014-cutoff, and removing duplicates:",
    final_unmatched_memb_rows_final.shape[0]
)
#display(final_unmatched_memb_rows_final.sort_values(by=["country_code_memb", "party_label_memb", "member_endDate_memb"]))

Final amount of unmatched 'rows' in membership dataset after mapping:  314
Amount of dropped rows because of removing 'independent': 126
Final amount of unmatched 'rows' in membership dataset after mapping and removing 'independent':  188
Amount of dropped rows because membership ended before 2014: 68
Final amount of unmatched 'rows' in membership dataset after mapping, removing 'independent', and 2014-cutoff: 120
Final amount of unmatched parties (ie. party_id_memb's) in membership dataset after mapping, removing 'independent', 2014-cutoff, and removing duplicates: 89


These national memberships/parties were just not present in our CHES dataset which is reasonable since a lot of minor political parties (e.g. the german 'ödp') are not included in the survey waves of CHES in general.

Ok, finally. Now store as enriched memberships dataframe

In [38]:
# remove unneeded columns
display(enriched_memberships_df.head(2))
enriched_memberships_df = enriched_memberships_df.drop(['label_norm_memb', 'name_norm_memb', 'label_norm_ches', 'name_norm_ches'], axis=1)
display(enriched_memberships_df.head(2))
#enriched_memberships_df.drop(['label_norm_memb', 'name_norm_memb', 'label_norm_ches	', 'name_norm_ches'], axis=1)

,person_id_memb,person_label_memb,hasMembership_memb,party_id_memb,member_startDate_memb,member_endDate_memb,member_role_memb,party_class_memb,country_code_memb,party_temporal_memb,party_label_memb,party_all_labels_memb,party_all_names_memb,label_norm_memb,name_norm_memb,party_id_ches,country_code_ches,party_abbrev_ches,party_name_ches,party_name_en_ches,label_norm_ches,name_norm_ches
0,102886,Ildikó GÁLL-PELCZ,"[{'id': 'membership/102886-f-149915', 'type': ...",4068,2014-07-01,2017-08-31,MEMBER,NATIONAL_POLITICAL_GROUP,HUN,"{'id': 'time-period/20140701-20190701', 'type'...",Fidesz-KDNP,[Fidesz-KDNP],[Fidesz-Magyar Polgári Szövetség-Kereszténydem...,{fidesz-kdnp},{fidesz-magyar polgari szovetseg-keresztenydem...,2302,HUN,"[Fidesz, Fidesz-KDNP, Fidesz-M]","[Fidesz—Magyar Polgári Szövetség, Fidesz—Keres...","[Fidesz—Hungarian Civic Union, Fidesz—Christia...","{fidesz, fidesz-kdnp, fidesz-m}","{fidesz—keresztenydemokrata neppart, fidesz—hu..."
1,102886,Ildikó GÁLL-PELCZ,"[{'id': 'membership/102886-f-149915', 'type': ...",2753,2010-06-02,2014-06-30,MEMBER,NATIONAL_POLITICAL_GROUP,HUN,"{'id': 'time-period/20090714-20140630', 'type'...",Fidesz-KDNP,[Fidesz-KDNP],[Fidesz-Magyar Polgári Szövetség-Keresztény De...,{fidesz-kdnp},{fidesz-magyar polgari szovetseg-kereszteny de...,2302,HUN,"[Fidesz, Fidesz-KDNP, Fidesz-M]","[Fidesz—Magyar Polgári Szövetség, Fidesz—Keres...","[Fidesz—Hungarian Civic Union, Fidesz—Christia...","{fidesz, fidesz-kdnp, fidesz-m}","{fidesz—keresztenydemokrata neppart, fidesz—hu..."


,person_id_memb,person_label_memb,hasMembership_memb,party_id_memb,member_startDate_memb,member_endDate_memb,member_role_memb,party_class_memb,country_code_memb,party_temporal_memb,party_label_memb,party_all_labels_memb,party_all_names_memb,party_id_ches,country_code_ches,party_abbrev_ches,party_name_ches,party_name_en_ches
0,102886,Ildikó GÁLL-PELCZ,"[{'id': 'membership/102886-f-149915', 'type': ...",4068,2014-07-01,2017-08-31,MEMBER,NATIONAL_POLITICAL_GROUP,HUN,"{'id': 'time-period/20140701-20190701', 'type'...",Fidesz-KDNP,[Fidesz-KDNP],[Fidesz-Magyar Polgári Szövetség-Kereszténydem...,2302,HUN,"[Fidesz, Fidesz-KDNP, Fidesz-M]","[Fidesz—Magyar Polgári Szövetség, Fidesz—Keres...","[Fidesz—Hungarian Civic Union, Fidesz—Christia..."
1,102886,Ildikó GÁLL-PELCZ,"[{'id': 'membership/102886-f-149915', 'type': ...",2753,2010-06-02,2014-06-30,MEMBER,NATIONAL_POLITICAL_GROUP,HUN,"{'id': 'time-period/20090714-20140630', 'type'...",Fidesz-KDNP,[Fidesz-KDNP],[Fidesz-Magyar Polgári Szövetség-Keresztény De...,2302,HUN,"[Fidesz, Fidesz-KDNP, Fidesz-M]","[Fidesz—Magyar Polgári Szövetség, Fidesz—Keres...","[Fidesz—Hungarian Civic Union, Fidesz—Christia..."


In [39]:
# store as parquet file
#enriched_memberships_df.to_parquet(PATH_INTERMED_CHES_53_ENRICHED_MEMBER, engine='fastparquet')